# [Walta News](http://www.waltainfo.com/index.php)

In [11]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd 
import re

In [15]:
def isEnglish(s):
    """Takes a token and checks if the input token is english or other language
        Input (str): input token
        output (bool): true if english else false"""
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return bool(re.findall('[A-Za-z]', s))
    else:
        return True


In [18]:
url = "http://www.waltainfo.com/index.php?locale=am"
crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
cookies = crawl_url.cookies

url_list = list()
# url_list.append({"category":"health", "url":"http://www.waltainfo.com/index.php/health?page=%s","max_page":25})
# url_list.append({"category":"technology", "url":"http://www.waltainfo.com/index.php/SATech?page=%s","max_page":32})
url_list.append({"category":"sport", "url":"http://www.waltainfo.com/index.php/EASport?page=%s","max_page":67})
url_list.append({"category":"featured", "url":"http://www.waltainfo.com/index.php/FeaturedArticles?page=%s","max_page":206})
url_list.append({"category":"national", "url":"http://www.waltainfo.com/index.php/news/national?page=%s","max_page":556})


def spiderWalta(max_pages, page, cookies=None):
    data = list()
    for i in tqdm(range(2, max_pages)):
        url = page%i
        if cookies != None:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, cookies=cookies)
        else:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        crawl_url.raise_for_status()
        soup = BeautifulSoup(crawl_url.text, 'lxml')
        body = soup.find("div", class_="media-body")
        if body != None:            
            title = body.h4.text
            title = " ".join(title.split())
            meta_data = body.find("div", class_="wic-content-desc-header")
            if meta_data != None:
                date = meta_data.find('span', class_="published-date").text
                author = meta_data.find('span', class_= "wic-content-auhtor-txt").text
            else:
                date, author = None, None
            content = " ".join(s for s in body.find("div", class_="wic-content-description").text.split() if not isEnglish(s))
            data.append([date, author, title, content])
        media = soup.find_all("div", class_="media")
        if media == None:
            continue
        links = [x.a['href'] for x in media if bool(x.findChildren("a", recursive=False))]
        for link in links:
            if cookies != None:
                crawl_url = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'}, cookies=cookies)
            else:
                crawl_url = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
            crawl_url.raise_for_status()
            soup = BeautifulSoup(crawl_url.text, 'lxml')
            body = soup.find("div", class_="media-body")
            if body == None:
                continue
            title = body.h4.text
            title = " ".join(title.split())
            meta_data = body.find("div", class_="wic-content-desc-header")
            if meta_data != None:
                date = meta_data.find('span', class_="published-date").text
                author = meta_data.find('span', class_= "wic-content-auhtor-txt").text
            else:
                date, author = None, None
            content = " ".join(s for s in body.find("div", class_="wic-content-description").text.split() if not isEnglish(s))
            data.append([date, author, title, content])
    return data

In [ ]:
for site in url_list:
    data = spiderWalta(site['max_page'], site['url'], cookies)
    df = pd.DataFrame(data, columns =['date', 'author', 'title', 'content']) 
    df.to_csv("walta_"+site['category']+".csv", encoding ='utf-16')

 54%|████████████████████████████████████████████▏                                     | 35/65 [07:23<06:15, 12.51s/it]